In [1]:
%pylab inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.tools.plotting import scatter_matrix
from scipy.stats import gaussian_kde
from pydoc import help
from scipy.stats.stats import pearsonr
from sklearn.metrics import explained_variance_score
from scipy.stats import wilcoxon
import warnings
import os
import glob
warnings.filterwarnings('ignore')
from __future__ import print_function, division

Populating the interactive namespace from numpy and matplotlib


In [2]:
direc = './'

# empty list to store the list of csv files
filelist =[]

# empty lists to put the results into
n_trees = []
r2_list = []
rmse_list = []
pRMSE_list = []
variance_list = []
Bias_list = []
n_samples =[]


# file pattern to search for in the directory.
pattern = "%sPredictedANDobserved_*" % direc

# return a list of files matching the pattern "csv" in the directry 
file1 = glob.glob(pattern)

for path in file1:
    a = path[2:]
    filelist.append(a)

# loop through the list of csv files and opens them as dataframes and selectes the rmse results and transforms them and returns them to a list
for filename in filelist:


    # import the raw results and calculate the statstics
    n_t = int(filename[22:-4])
    n_trees.append(n_t)
    dfa = pd.read_csv(filename, header=None)# reads the csv file
    # sort out the outputs into two colums of data to calculate the stats
    pred =  dfa[dfa.columns[0:5]]
    obs = dfa[dfa.columns[5:]]
    
    # stack the results of the 20 runs on top of each other 
    st_pred = pred.stack()
    st_obs = obs.stack()
    
    a = st_pred.values
    b = st_obs.values
    
    df = pd.DataFrame([a,b]).T
    df.columns=['predicted','observed']
    
    # selects the model using the spectral predictor variables 
    y = df.observed.values
    x = df.predicted.values
    df['res_L5'] = df['predicted'] - df['observed']
    sumRes = df['res_L5'].sum()
    """
    code to calculate the pearson's correlation coefficient and the coefficient of determination, 
    the for loop extracts out the correlatin coeffiecent score which is a tuple containing the R and p value 
    """
    r = pearsonr(x,y)
    r2 = r[0:1]
    personsr = []
    for i in r:
        personsr.append(i)
    r2 = personsr[0]**2
    r2s = str(r2)

    # calculates the variance 
    meanN13error = df['res_L5'].mean()
    n1 = len(df['predicted'])-1 
    df['var1_l5'] = (df['res_L5']-meanN13error)**2
    vari_n13 = str(df['var1_l5'].sum()/n1)

    # formats the stats into strings to enable them to be put in as text in the plot
    r2s = str(format(r2, '.2f'))
    r2_list.append(r2s)
    rmseS = str(format(np.sqrt(np.mean((y - x) ** 2)), '.3f'))
    rmse_list.append(rmseS)
    var_n13 = str(format(df['var1_l5'].sum()/n1,'.3f'))
    variance_list.append(var_n13)
    biasS = str(format(sumRes /len(y), '.3f'))
    Bias_list.append(biasS)
    nS = str(len(x))
    n_samples.append(nS)
    # calculate the RMSE expressed as a percentage of the observed mean (RMSE%) for each response variable.

    rRMSE = str(format((np.sqrt(np.mean((y - x) ** 2))/np.mean(y))*100,'.2f'))
    pRMSE_list.append(rRMSE)
    
# create a data frame with the results and export to csv
df_res = pd.DataFrame([n_trees,r2_list,rmse_list, pRMSE_list,variance_list, Bias_list, n_samples]).T
df_res.columns= ['n_trees', 'R2', 'RMSE', 'RMSE%','Variance','Bias','n_samples']
clean_res = df_res.sort_values(by=['n_trees'])
clean_res.to_csv("s2_10m_varReduction_results_h99_CanopyHeight.csv")



In [3]:
#results of the n trees analysis 
clean_res

,n_trees,R2,RMSE,RMSE%,Variance,Bias,n_samples
0,1,0.42,4.913,43.00,24.137,0.004,74000
11,2,0.60,4.084,35.75,16.679,0.012,74000
22,3,0.67,3.711,32.49,13.774,0.020,74000
33,4,0.69,3.572,31.23,12.759,-0.004,74000
44,5,0.70,3.530,31.00,12.463,0.020,74000
49,6,0.73,3.351,29.29,11.226,-0.007,74000
50,7,0.74,3.291,28.72,10.828,-0.004,74000
51,8,0.74,3.264,28.63,10.657,0.010,74000
52,9,0.74,3.270,28.59,10.692,0.006,74000
1,10,0.75,3.223,28.18,10.386,0.013,74000


In [7]:
# read in the selected n_trees band importance scores list
impS = pd.read_csv('FeatureImportance_Band_Score_n24.csv',header=0)

In [8]:
list_var = (impS['band'].values)
print (len(list_var))
list_var

24


array(['NDRE1d', 'ratio62a', 'CVIa', 'ratio65a', 'GNDVIa', 'ratio63d',
       'CVId', 'ratio95d', 'psB9d', 'ratio98a', 'ratio62d', 'ratio54a',
       'ratio92a', 'ratio54d', 'ratio105a', 'psB10a', 'SAVIa', 'NDIId',
       'psB3d', 'ratio103d', 'psB2a', 'ratio103a', 'ratio107d', 'ratio109d'], dtype=object)

In [9]:
list_var = (impS['band'].values)
print (len(list_var))
list_var

24


array(['NDRE1d', 'ratio62a', 'CVIa', 'ratio65a', 'GNDVIa', 'ratio63d',
       'CVId', 'ratio95d', 'psB9d', 'ratio98a', 'ratio62d', 'ratio54a',
       'ratio92a', 'ratio54d', 'ratio105a', 'psB10a', 'SAVIa', 'NDIId',
       'psB3d', 'ratio103d', 'psB2a', 'ratio103a', 'ratio107d', 'ratio109d'], dtype=object)